In [1]:
# import all the junk I need
import numpy as np
from skimage import io, data
import scipy
from scipy import signal
import matplotlib.pyplot as plt
import mrc
import os
import re
import csv
from natsort import natsorted
import pandas as pd
from scipy import stats as st


In [42]:
### define functions ###

# fix inconsistent filenames TT
def rename_files_in_folder(folder):
    for filename in os.listdir(folder):
        if os.path.isfile(os.path.join(folder, filename)):
            new_filename = fixnames(filename)
            old_filepath = os.path.join(folder, filename)
            new_filepath = os.path.join(folder, new_filename)
            os.rename(old_filepath, new_filepath)
            print(f"Renamed {filename} to {new_filename}")

def fixnames(filename):
    pattern = r"_pt\d+"  # Matches "_pt" followed by one or more digits
    new_filename = re.sub(pattern, "", filename)
    return new_filename



# look at files in a folder and find .dvs with ref in the appropriate position + matched non ref
def organize(folder):
    list = natsorted(os.listdir(folder))
    cilialist = []
    reflist = []
    for m in list:
        if m[-3:] == '.dv' and m[24:27] != 'ref':
            cilialist.append(m)
        else:
            if m[-3:] == '.dv' and m[24:27] == 'ref':
                reflist.append(m)
            else:
                pass
    return cilialist, reflist  

# calculate power and dominant frequency for each pixel
def powerfreq_all(pix):
    f, pxx = scipy.signal.periodogram(pix, fs=125)
    max_pxx = np.max(pxx)
    index = np.where(pxx == max_pxx)
    domfreq = f[index]
    return domfreq[0], max_pxx

### i don't end up using this one but it could be handy someday
# calculate power and freq for a whole image at once
def powerfreq_whole(image):
    f, pxx = signal.periodogram(image, fs=125, axis=0)
    max_pxx = np.max(pxx, axis=0)
    index = np.argmax(pxx, axis=0)
    domfreq = f[index]
    return domfreq[0], max_pxx

def perpair_processing(folder):
    path = folder + 'processed/'
    ciliavids, refimgs = organize(folder)

    for x in range(len(ciliavids)):
        current = ciliavids[x]
        if x < len(refimgs):
            refname = refimgs[x]
        else:
            refname = None
        print('Processing: ' + current + refname)

        # Read cilia video
        image = mrc.imread(folder + current)
        rows, cols = image.shape[1:]
        
        # Read matched reference image
        if refname:
            refimage = mrc.imread(folder + refname)
            refproj = refimage.max(axis=1)
            refcent = refproj[:, 383:639, 383:639]
        else:
            refcent = np.empty((4, rows, cols))
            refcent.fill(np.nan)

        # Create empty arrays for storing pixel values
        power = np.empty((rows, cols))
        domfreq = np.empty((rows, cols))
        sumproj = np.empty((rows, cols))
        refintensity = np.empty((4, rows, cols))

        # Process each pixel
        for i in range(rows):
            for j in range(cols):
                domfreq[i, j], power[i, j] = powerfreq_all(image[:, i, j])
                sumproj[i, j] = image[:, i, j].sum()
                refintensity[:, i, j] = refcent[:, i, j]

        # Save power, dominant frequency, sum projection, and reference image intensity as TIFF
        output_tiff = np.concatenate((power[np.newaxis, :, :], domfreq[np.newaxis, :, :],
                                      sumproj[np.newaxis, :, :], refintensity), axis=0)
        tiff_filename = folder + 'processed/' + current[:-3] + '_nugget.tif'
        io.imsave(tiff_filename, output_tiff, check_contrast=False)
        print('Saved: ' + tiff_filename)

        # Save pixel values as CSV
        output_csv = np.column_stack((np.repeat(current, rows*cols),
                                      np.repeat(refname, rows*cols),
                                      np.repeat(np.arange(rows), cols),
                                      np.tile(np.arange(cols), rows),
                                      power.ravel(), domfreq.ravel(), sumproj.ravel(),
                                      refintensity.reshape(4, -1).T))
        csv_filename = folder + 'processed/' + current[:-3] + '_perpixsummary.csv'
        with open(csv_filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['video', 'refimage', 'i', 'j', 'power', 'domfreq', 'sumproj',
                             'spytub', 'nucview', 'gfp', 'pol'])
            writer.writerows(output_csv)
        print('Saved: ' + csv_filename)

    return

In [64]:
##### run this #####
folder = '/Users/Confocal/Desktop/ciliasinglets/'
test = perpair_processing(folder)


Processing: mb_220516_cilia_t0_p1_01_R3D.dv
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p1_01_R3D_nugget.tif
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p1_01_R3D_perpixsummary.csv
Processing: mb_220516_cilia_t0_p1_02_R3D.dv
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p1_02_R3D_nugget.tif
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p1_02_R3D_perpixsummary.csv
Processing: mb_220516_cilia_t0_p1_03_R3D.dv
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p1_03_R3D_nugget.tif
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p1_03_R3D_perpixsummary.csv
Processing: mb_220516_cilia_t0_p1_04_R3D.dv
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p1_04_R3D_nugget.tif
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p1_04_R3D_perpixsummary.csv
Processing: mb_220516_cilia_t0_p1_05

Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p4_05_R3D_nugget.tif
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p4_05_R3D_perpixsummary.csv
Processing: mb_220516_cilia_t0_p4_06_R3D.dv
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p4_06_R3D_nugget.tif
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p4_06_R3D_perpixsummary.csv
Processing: mb_220516_cilia_t0_p4_07_R3D.dv
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p4_07_R3D_nugget.tif
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p4_07_R3D_perpixsummary.csv
Processing: mb_220516_cilia_t0_p4_08_R3D.dv
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p4_08_R3D_nugget.tif
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p4_08_R3D_perpixsummary.csv
Processing: mb_220516_cilia_t0_p4_09_R3D.dv
Saved: /Users/Confocal/Desktop/cilia

Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p7_08_R3D_nugget.tif
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p7_08_R3D_perpixsummary.csv
Processing: mb_220516_cilia_t0_p7_09_R3D.dv
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p7_09_R3D_nugget.tif
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p7_09_R3D_perpixsummary.csv
Processing: mb_220516_cilia_t0_p7_10_R3D.dv
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p7_10_R3D_nugget.tif
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p7_10_R3D_perpixsummary.csv
Processing: mb_220516_cilia_t0_p8_01_R3D.dv
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p8_01_R3D_nugget.tif
Saved: /Users/Confocal/Desktop/ciliasinglets/processed/mb_220516_cilia_t0_p8_01_R3D_perpixsummary.csv
Processing: mb_220516_cilia_t0_p8_02_R3D.dv
Saved: /Users/Confocal/Desktop/cilia

In [43]:
folder = 'D:/ruxcilia/cilia/'
test = perpair_processing(folder)

Processing: mb_220616_ruxcilia_t3cilia_08_R3D.dvmb_220616_ruxcilia_t310xrefs_008_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_08_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_08_R3D_perpixsummary.csv
Processing: mb_220616_ruxcilia_t3cilia_09_R3D.dvmb_220616_ruxcilia_t310xrefs_009_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_09_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_09_R3D_perpixsummary.csv
Processing: mb_220616_ruxcilia_t3cilia_10_R3D.dvmb_220616_ruxcilia_t310xrefs_010_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_10_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_10_R3D_perpixsummary.csv
Processing: mb_220616_ruxcilia_t3cilia_11_R3D.dvmb_220616_ruxcilia_t310xrefs_011_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_11_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_11_R3

Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_40_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_40_R3D_perpixsummary.csv
Processing: mb_220616_ruxcilia_t3cilia_41_R3D.dvmb_220616_ruxcilia_t310xrefs_041_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_41_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_41_R3D_perpixsummary.csv
Processing: mb_220616_ruxcilia_t3cilia_42_R3D.dvmb_220616_ruxcilia_t310xrefs_042_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_42_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_42_R3D_perpixsummary.csv
Processing: mb_220616_ruxcilia_t3cilia_43_R3D.dvmb_220616_ruxcilia_t310xrefs_043_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_43_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_43_R3D_perpixsummary.csv
Processing: mb_220616_ruxcilia_t3cilia_44_R3D.dvmb_220616_ruxcilia_t

Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_72_R3D_perpixsummary.csv
Processing: mb_220616_ruxcilia_t3cilia_73_R3D.dvmb_220616_ruxcilia_t310xrefs_073_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_73_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_73_R3D_perpixsummary.csv
Processing: mb_220616_ruxcilia_t3cilia_74_R3D.dvmb_220616_ruxcilia_t310xrefs_074_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_74_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_74_R3D_perpixsummary.csv
Processing: mb_220616_ruxcilia_t3cilia_75_R3D.dvmb_220616_ruxcilia_t310xrefs_075_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_75_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_75_R3D_perpixsummary.csv
Processing: mb_220616_ruxcilia_t3cilia_76_R3D.dvmb_220616_ruxcilia_t310xrefs_076_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cili

Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_105_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_105_R3D_perpixsummary.csv
Processing: mb_220616_ruxcilia_t3cilia_106_R3D.dvmb_220616_ruxcilia_t310xrefs_106_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_106_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_106_R3D_perpixsummary.csv
Processing: mb_220616_ruxcilia_t3cilia_107_R3D.dvmb_220616_ruxcilia_t310xrefs_107_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_107_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_107_R3D_perpixsummary.csv
Processing: mb_220616_ruxcilia_t3cilia_108_R3D.dvmb_220616_ruxcilia_t310xrefs_108_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_108_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_108_R3D_perpixsummary.csv
Processing: mb_220616_ruxcilia_t3cilia_109_R3D.dvmb_22061

Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_137_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_137_R3D_perpixsummary.csv
Processing: mb_220616_ruxcilia_t3cilia_138_R3D.dvmb_220616_ruxcilia_t310xrefs_138_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_138_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_138_R3D_perpixsummary.csv
Processing: mb_220616_ruxcilia_t3cilia_139_R3D.dvmb_220616_ruxcilia_t310xrefs_139_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_139_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_139_R3D_perpixsummary.csv
Processing: mb_220616_ruxcilia_t3cilia_140_R3D.dvmb_220616_ruxcilia_t310xrefs_140_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_140_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220616_ruxcilia_t3cilia_140_R3D_perpixsummary.csv
Processing: mb_220616_ruxcilia_t3cilia_141_R3D.dvmb_22061

...

Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_18_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_18_R3D_perpixsummary.csv
Processing: mb_220618_ruxcilia_t7cilia_19_R3D.dvmb_220619_ruxcilia_t710xrefs_019_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_19_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_19_R3D_perpixsummary.csv
Processing: mb_220618_ruxcilia_t7cilia_20_R3D.dvmb_220619_ruxcilia_t710xrefs_020_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_20_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_20_R3D_perpixsummary.csv
Processing: mb_220618_ruxcilia_t7cilia_21_R3D.dvmb_220619_ruxcilia_t710xrefs_021_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_21_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_21_R3D_perpixsummary.csv
Processing: mb_220618_ruxcilia_t7cilia_22_R3D.dvmb_220619_ruxcilia_t

Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_50_R3D_perpixsummary.csv
Processing: mb_220618_ruxcilia_t7cilia_51_R3D.dvmb_220619_ruxcilia_t710xrefs_051_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_51_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_51_R3D_perpixsummary.csv
Processing: mb_220618_ruxcilia_t7cilia_52_R3D.dvmb_220619_ruxcilia_t710xrefs_052_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_52_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_52_R3D_perpixsummary.csv
Processing: mb_220618_ruxcilia_t7cilia_53_R3D.dvmb_220619_ruxcilia_t710xrefs_053_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_53_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_53_R3D_perpixsummary.csv
Processing: mb_220618_ruxcilia_t7cilia_54_R3D.dvmb_220619_ruxcilia_t710xrefs_054_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cili

Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_83_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_83_R3D_perpixsummary.csv
Processing: mb_220618_ruxcilia_t7cilia_84_R3D.dvmb_220619_ruxcilia_t710xrefs_084_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_84_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_84_R3D_perpixsummary.csv
Processing: mb_220618_ruxcilia_t7cilia_85_R3D.dvmb_220619_ruxcilia_t710xrefs_085_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_85_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_85_R3D_perpixsummary.csv
Processing: mb_220618_ruxcilia_t7cilia_86_R3D.dvmb_220619_ruxcilia_t710xrefs_086_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_86_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_86_R3D_perpixsummary.csv
Processing: mb_220618_ruxcilia_t7cilia_87_R3D.dvmb_220619_ruxcilia_t

Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_115_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_115_R3D_perpixsummary.csv
Processing: mb_220618_ruxcilia_t7cilia_116_R3D.dvmb_220619_ruxcilia_t710xrefs_116_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_116_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_116_R3D_perpixsummary.csv
Processing: mb_220618_ruxcilia_t7cilia_117_R3D.dvmb_220619_ruxcilia_t710xrefs_117_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_117_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_117_R3D_perpixsummary.csv
Processing: mb_220618_ruxcilia_t7cilia_118_R3D.dvmb_220619_ruxcilia_t710xrefs_118_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_118_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_118_R3D_perpixsummary.csv
Processing: mb_220618_ruxcilia_t7cilia_119_R3D.dvmb_22061

Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_147_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_147_R3D_perpixsummary.csv
Processing: mb_220618_ruxcilia_t7cilia_148_R3D.dvmb_220619_ruxcilia_t710xrefs_148_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_148_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_148_R3D_perpixsummary.csv
Processing: mb_220618_ruxcilia_t7cilia_149_R3D.dvmb_220619_ruxcilia_t710xrefs_149_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_149_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_149_R3D_perpixsummary.csv
Processing: mb_220618_ruxcilia_t7cilia_150_R3D.dvmb_220619_ruxcilia_t710xrefs_150_R3D.dv
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_150_R3D_nugget.tif
Saved: D:/ruxcilia/cilia/processed/mb_220618_ruxcilia_t7cilia_150_R3D_perpixsummary.csv
Processing: mb_220618_ruxcilia_t7cilia_151_R3D.dvmb_22061

120

In [54]:
### concatenate the csvs and add extra info ###

def concatenate_csvs(folder, mapping_csv, timepoints_csv):
    # Get a list of all CSV files in the folder
    csv_files = [file for file in os.listdir(folder) if file.endswith('.csv')]

    # Read the mapping CSV files
    mapping_df = pd.read_csv(mapping_csv)
    timepoints_df = pd.read_csv(timepoints_csv)
    
    # Create an empty list to store the concatenated dataframes
    dfs = []

    # Iterate over the CSV files
    for file in csv_files:
        # Read each CSV file
        csv_path = os.path.join(folder, file)
        df = pd.read_csv(csv_path)

        # Extract the file information from the filename
        filename = os.path.splitext(file)[0]
        timepoint = int(filename.split('_')[3][1:2])
        point = int(filename.split('_')[4])

        # Lookup donor and mock/infected information from the mapping CSV
        donor = mapping_df.loc[mapping_df['vidpoint'] == point, 'donor'].values[0]
        mock_or_infected = mapping_df.loc[mapping_df['vidpoint'] == point, 'condition'].values[0]
        culturepoint = mapping_df.loc[mapping_df['vidpoint'] == point, 'culturepoint'].values[0]
        experiment = mapping_df.loc[mapping_df['vidpoint'] == point, 'experiment'].values[0]
        
        # Lookup hpi from other mapping csv
        hpi = timepoints_df.loc[timepoints_df['timed_point'] == timepoint, 'hpi'].values[0]
        
        # Add the additional columns
        df['timepoint'] = timepoint
        df['hpi'] = hpi
        df['donor'] = donor
        df['vidpoint'] = point
        df['condition'] = mock_or_infected
        df['culturepoint'] = culturepoint
        df['experiment'] = experiment
        # Append the dataframe to the list
        dfs.append(df)

    # Concatenate the dataframes
    result_df = pd.concat(dfs)

    # Save the concatenated dataframe to a new CSV file
    output_csv = os.path.join(folder, 'concatenated.csv')
    result_df.to_csv(output_csv, index=False)

    print('Concatenated CSV saved as: ' + output_csv)


# Usage
#folder = '/Users/Confocal/Desktop/ciliabulk2/processed/'
#mapping_csv = '/Users/Confocal/Desktop/ciliabulk2/cilia_ptskey.csv'
#timepoints_csv = '/Users/Confocal/Desktop/ciliabulk2/timepoints.csv'

folder = 'D:/ruxcilia/cilia/processed/'
mapping_csv = 'D:/ruxcilia/ruxcilia_ptskey.csv'
timepoints_csv = 'D:/ruxcilia/ruxcilia_timepoints.csv'


concatenate_csvs(folder, mapping_csv, timepoints_csv)


Concatenated CSV saved as: D:/ruxcilia/cilia/processed/concatenated.csv


In [52]:
csv_files = [file for file in os.listdir(folder) if file.endswith('.csv')]
file = csv_files[0]

filename = os.path.splitext(file)[0]
point = int(filename.split('_')[4])
point
    
mapping_csv = 'D:/ruxcilia/ruxcilia_ptskey.csv'
mapping_df = pd.read_csv(mapping_csv)
donor = mapping_df.loc[mapping_df['vidpoint'] == point, 'donor'].values[0]

donor

3

In [40]:
def organize(folder):
    list = natsorted(os.listdir(folder))
    cilialist = []
    reflist = []
    for m in list:
        if m[-3:] == '.dv' and m[24:27] != 'ref':
            cilialist.append(m)
        else:
            if m[-3:] == '.dv' and m[24:27] == 'ref':
                reflist.append(m)
            else:
                pass
    return cilialist, reflist  

folder = 'D:/ruxcilia/cilia/'

mismatch = organize(folder)
csv_filename = folder + 'mismatches.csv'
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['video', 'refimage'])
    writer.writerows(mismatch)
print('Saved: ' + csv_filename)

        
        

Saved: D:/ruxcilia/cilia/mismatches.csv


In [24]:
folder = 'D:/ruxcilia/cilia/processed/'
csv_files = [file for file in os.listdir(folder) if file.endswith('.csv')]
file = csv_files[1]

filename = os.path.splitext(file)[0]
timepoint = int(filename.split('_')[3][1:2])
point = int(filename.split('_')[4])

csv_filename = folder + 'processed/' + current[:-3] + '_perpixsummary.csv'
        with open(csv_filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['video', 'refimage', 'i', 'j', 'power', 'domfreq', 'sumproj',
                             'spytub', 'nucview', 'gfp', 'pol'])
            writer.writerows(output_csv)
        print('Saved: ' + csv_filename)

        
        

['mb_220616_ruxcilia_t3cilia_139_R3D_perpixsummary.csv',
 'mb_220616_ruxcilia_t3cilia_140_R3D_perpixsummary.csv',
 'mb_220616_ruxcilia_t3cilia_141_R3D_perpixsummary.csv',
 'mb_220616_ruxcilia_t3cilia_142_R3D_perpixsummary.csv',
 'mb_220616_ruxcilia_t3cilia_143_R3D_perpixsummary.csv',
 'mb_220616_ruxcilia_t3cilia_144_R3D_perpixsummary.csv',
 'mb_220616_ruxcilia_t3cilia_145_R3D_perpixsummary.csv',
 'mb_220616_ruxcilia_t3cilia_146_R3D_perpixsummary.csv',
 'mb_220616_ruxcilia_t3cilia_147_R3D_perpixsummary.csv',
 'mb_220616_ruxcilia_t3cilia_148_R3D_perpixsummary.csv',
 'mb_220616_ruxcilia_t3cilia_149_R3D_perpixsummary.csv',
 'mb_220616_ruxcilia_t3cilia_150_R3D_perpixsummary.csv',
 'mb_220616_ruxcilia_t3cilia_151_R3D_perpixsummary.csv',
 'mb_220616_ruxcilia_t3cilia_152_R3D_perpixsummary.csv',
 'mb_220616_ruxcilia_t3cilia_153_R3D_perpixsummary.csv',
 'mb_220616_ruxcilia_t3cilia_154_R3D_perpixsummary.csv',
 'mb_220616_ruxcilia_t3cilia_155_R3D_perpixsummary.csv',
 'mb_220616_ruxcilia_t3cilia_15

In [55]:
##### testing csv processing #####
folder = '/Users/Confocal/Desktop/ciliabulk2/small/'
mapping_csv = '/Users/Confocal/Desktop/ciliabulk2/cilia_ptskey.csv'
csv_files = [file for file in os.listdir(folder) if file.endswith('.csv')]
mapping_df = pd.read_csv(mapping_csv)

    # Create an empty list to store the concatenated dataframes
dfs = []

    # Iterate over the CSV files
for file in csv_files:
       # Read each CSV file
    csv_path = os.path.join(folder, file)
    df = pd.read_csv(csv_path)
        # Extract the file information from the filename
    filename = os.path.splitext(file)[0]
    timepoint = int(filename.split('_')[3][1:])
    point = int(filename.split('_')[6])
    mapping_df.loc[mapping_df['vidpoint'] == point, 'culturepoint'].values[0]
    #print(filename)
        # Lookup donor and mock/infected information from the mapping CSV
#        donor = mapping_df.loc[mapping_df['Point'] == point, 'Donor'].values[0]
#        mock_or_infected = mapping_df.loc[mapping_df['Point'] == point, 'Mock_Infected'].values[0]


In [54]:
point = '1'
[mapping_df['culturepoint'] == point, 'culturepoint']

[0     False
 1     False
 2     False
 3     False
 4     False
       ...  
 86    False
 87    False
 88    False
 89    False
 90    False
 Name: culturepoint, Length: 91, dtype: bool,
 'culturepoint']

In [ ]:
##### TESTING IMAGE PROCESSING #####
folder = '/Users/Confocal/Desktop/ciliabulk2/'
ciliavids, refimgs = organize(folder)
current = ciliavids[0]
image = mrc.imread(folder + current)
rows, cols = image.shape[1:]
refname = refimgs[0]
refimage = mrc.imread(folder + refname)
refproj = refimage.max(axis=1)
refcent = refproj[:, 383:639, 383:639]

power = np.empty((rows, cols))
domfreq = np.empty((rows, cols))
sumproj = np.empty((rows, cols))
refintensity = np.empty((4, rows, cols))
for i in range(rows):
    for j in range(cols):
        domfreq[i, j], power[i, j] = powerfreq_all(image[:, i, j])
        sumproj[i, j] = image[:, i, j].sum()
        refintensity[:, i, j] = refcent[:, i, j]

output_tiff = np.concatenate((power[np.newaxis, :, :], domfreq[np.newaxis, :, :],
                                      sumproj[np.newaxis, :, :], refintensity), axis=0)
    

In [ ]:
##### MORE TESTING #####
#io.imshow(refintensity[0])
#refintensity.ravel()
test = refintensity.reshape(4, -1)
#refintensity.shape
test[0, 3]
